### Initial Multitask Model

#### Import Necessary Libraries

In [1]:
import pandas as pd
import torch
import torch.nn as nn # neural network layers and modules
import torch.optim as optim # optimisers
from torch.utils.data import DataLoader, Dataset # data batching tools
from transformers import T5Tokenizer, T5ForConditionalGeneration # huggingface T5 model + tokenizer
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix # sklearn metrics for evaluation
import torchmetrics # extra metrics
from model import MultitaskModel, MultitaskDataset, train, evaluate
from sklearn.model_selection import train_test_split

### Import Dataset

In [2]:
mwr_df_simple = pd.read_csv('mwr_simple.csv')

### Extract Features

In [3]:
print(mwr_df_simple.columns)

Index(['Examination ID', 'Conclusion', 'r:Th', 'Weight', 'Height',
       'Ambient temperature', 'r:AgeInYears', 'Mammary diameter', 'Cycle',
       'Day from the first day', 'Hormonal medications',
       'Cancer family history', 'Breast operations', 'Num of pregnancies',
       'R1 int', 'L1 int', 'R2 int', 'L2 int', 'R3 int', 'L3 int', 'R4 int',
       'L4 int', 'R5 int', 'L5 int', 'R6 int', 'L6 int', 'R7 int', 'L7 int',
       'R8 int', 'L8 int', 'R9 int', 'L9 int', 'T1 int', 'T2 int', 'R0 int',
       'L0 int', 'R1 sk', 'L1 sk', 'R2 sk', 'L2 sk', 'R3 sk', 'L3 sk', 'R4 sk',
       'L4 sk', 'R5 sk', 'L5 sk', 'R6 sk', 'L6 sk', 'R7 sk', 'L7 sk', 'R8 sk',
       'L8 sk', 'R9 sk', 'L9 sk', 'T1 sk', 'T2 sk', 'R0 sk', 'L0 sk',
       'Conclusion (Tr)', 'Synthetic_Conclusion'],
      dtype='object')


In [4]:
# Select feature columns (sensor readings)
feature_cols = [col for col in mwr_df_simple.columns if col.endswith('int') or col.endswith('sk')]
mwr_df_simple['features'] = mwr_df_simple[feature_cols].values.tolist()

# Prepare labels and text targets
mwr_df_simple['class_label'] = mwr_df_simple['r:Th'].astype(int)
mwr_df_simple['synthetic_description'] = mwr_df_simple['Synthetic_Conclusion']

### Split Data into Test and Train

In [5]:
# Take a small sample for CPU testing
mwr_df_simple = mwr_df_simple.sample(n=200, random_state=42)

In [6]:
# 70% training, 15% validation, 15% testing
train_simple_df, temp_simple_df = train_test_split(mwr_df_simple, test_size=0.3, random_state=42)
val_simple_df, test_simple_df = train_test_split(temp_simple_df, test_size=0.5, random_state=42)

### Model Initialisation

In [ ]:
# Initialize tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Create datasets
train_dataset = MultitaskDataset(train_simple_df, tokenizer)
val_dataset = MultitaskDataset(val_simple_df, tokenizer)
test_dataset = MultitaskDataset(test_simple_df, tokenizer)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model and optimizer
model = MultitaskModel(num_classes=6).to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-5)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### Train Model

In [8]:
num_epochs = 10  # or whatever you choose

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    
    # Train
    train_loss = train(model, train_loader, optimizer, device)
    print(f"Training loss: {train_loss:.4f}")
    
    # Validate
    val_loss, val_metrics = evaluate(model, val_loader, device)
    print(f"Validation loss: {val_loss:.4f}")
    print(f"Validation metrics: {val_metrics}")



Epoch 1/10


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Training loss: 13.8391
Validation loss: 10.2960
Validation metrics: {'accuracy': 0.1, 'f1_score': 0.019354838709677417}

Epoch 2/10
Training loss: 10.8135
Validation loss: 7.0710
Validation metrics: {'accuracy': 0.1, 'f1_score': 0.01818181818181818}

Epoch 3/10
Training loss: 8.7286
Validation loss: 4.2547
Validation metrics: {'accuracy': 0.1, 'f1_score': 0.01875}

Epoch 4/10
Training loss: 6.3370
Validation loss: 2.6854
Validation metrics: {'accuracy': 0.13333333333333333, 'f1_score': 0.07268817204301076}

Epoch 5/10
Training loss: 4.7653
Validation loss: 2.0857
Validation metrics: {'accuracy': 0.06666666666666667, 'f1_score': 0.014285714285714285}

Epoch 6/10
Training loss: 3.7458
Validation loss: 2.0155
Validation metrics: {'accuracy': 0.13333333333333333, 'f1_score': 0.0707070707070707}

Epoch 7/10
Training loss: 3.4318
Validation loss: 2.0085
Validation metrics: {'accuracy': 0.06666666666666667, 'f1_score': 0.016}

Epoch 8/10
Training loss: 3.0601
Validation loss: 2.0152
Validatio

In [9]:
# Final evaluation on test set
test_loss, test_metrics = evaluate(model, test_loader, device)
print(f"Test loss: {test_loss:.4f}")
print(f"Test metrics: {test_metrics}")

Test loss: 2.0423
Test metrics: {'accuracy': 0.06666666666666667, 'f1_score': 0.10959595959595961}
